In [1]:
from ctypes import *
import math
import random
import os
import cv2
import numpy as np
import time
import darknet
import matplotlib.pyplot as plt

In [2]:
# opencv python 에서는 rect2D가 없어서 별도로 합집합 / 교집합을 구해야함.
#합집합
def union(a,b):
    x = min(a[0], b[0])
    y = min(a[1], b[1])
    w = max(a[0]+a[2], b[0]+b[2]) - x
    h = max(a[1]+a[3], b[1]+b[3]) - y
    return (x, y, w, h)
#교집합
def intersection(a,b):
    x = max(a[0], b[0])
    y = max(a[1], b[1])
    w = min(a[0]+a[2], b[0]+b[2]) - x
    h = min(a[1]+a[3], b[1]+b[3]) - y
    if w<0 or h<0: return () # or (0,0,0,0) ?
    return (x, y, w, h)

In [3]:
# 교차하는 부분을 찾기
def calculateIntersection(a0, a1, b0, b1):
    if a0 >= b0 and a1 <= b1: # Contained
        intersection = a1 - a0
    elif a0 < b0 and a1 > b1: # Contains
        intersection = b1 - b0
    elif a0 < b0 and a1 > b0: # Intersects right
        intersection = a1 - b0
    elif a1 > b1 and a0 < b1: # Intersects left
        intersection = b1 - a0
    else: # No intersection (either side)
        intersection = 0

    return intersection

In [4]:
def convertBack(x, y, w, h):
    xmin = int(round(x - (w / 2)))
    xmax = int(round(x + (w / 2)))
    ymin = int(round(y - (h / 2)))
    ymax = int(round(y + (h / 2)))
    return xmin, ymin, xmax, ymax

def cvDrawBoxes(detections, img):
#     print('detections length : ', len(detections))
    pt1 = (0, 0)
    pt2 = (0, 0)
    area = 0
    
    xmin = 0
    ymin = 0
    xmax = 0
    ymax = 0
    for detection in detections:
        if detection[0].decode() != 'car' and detection[0].decode() != 'bus' and detection[0].decode() != 'truck':
            continue

        x, y, w, h = detection[2][0],\
            detection[2][1],\
            detection[2][2],\
            detection[2][3]
        xmin, ymin, xmax, ymax = convertBack( float(x), float(y), float(w), float(h))
        pt1 = (xmin, ymin)
        pt2 = (xmax, ymax)
        cv2.rectangle(img, pt1, pt2, (0, 255, 0), 3)
#         cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), -1)
        cv2.putText(img,
                    detection[0].decode() +
                    " [" + str(round(detection[1] * 100, 2)) + "]",
                    (pt1[0], pt1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    [0, 255, 0], 2)
        area = (ymax-ymin) * (xmax-xmin)
    return img, area, (xmin, ymin, xmax, ymax)

outputWidth = 1920
outputHeight = 1080
netMain = None
metaMain = None
altNames = None 

In [8]:
global metaMain, netMain, altNames
configPath = "./cfg/yolov3-spp.cfg"
weightPath = "./yolov3-spp.weights"
metaPath = "./cfg/coco.data"
if not os.path.exists(configPath):
    raise ValueError("Invalid config path `" +
                     os.path.abspath(configPath)+"`")
if not os.path.exists(weightPath):
    raise ValueError("Invalid weight path `" +
                     os.path.abspath(weightPath)+"`")
if not os.path.exists(metaPath):
    raise ValueError("Invalid data file path `" +
                     os.path.abspath(metaPath)+"`")
if netMain is None:
    netMain = darknet.load_net_custom(configPath.encode(
        "ascii"), weightPath.encode("ascii"), 0, 1)  # batch size = 1
if metaMain is None:
    metaMain = darknet.load_meta(metaPath.encode("ascii"))
if altNames is None:
    try:
        with open(metaPath) as metaFH:
            metaContents = metaFH.read()
            import re
            match = re.search("names *= *(.*)$", metaContents,
                              re.IGNORECASE | re.MULTILINE)
            if match:
                result = match.group(1)
            else:
                result = None
            try:
                if os.path.exists(result):
                    with open(result) as namesFH:
                        namesList = namesFH.read().strip().split("\n")
                        altNames = [x.strip() for x in namesList]
            except TypeError:
                pass
    except Exception:
        pass
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("./photi-vision-data/parking03_edit.mp4")
cap.set(3, outputWidth)
cap.set(4, outputHeight)
out = cv2.VideoWriter(
    "./photi-vision-data/output-test2.mp4", cv2.VideoWriter_fourcc(*"MP4V"), 30.0,
    (outputWidth, outputHeight))
print("Starting the YOLO loop...")

#darknet.set_gpu(0)
# Create an image we reuse for each detect
darknet_image = darknet.make_image(outputWidth,
                                   outputHeight,3)

point1 = (0, 0)
point2 = (0, 0)
A = (point1, point2)
AREA = 1
X0, Y0, X1, Y1 = [0,0,0,0]
rectangleA = (0, 0, 0, 0)

temp_images = []
temp_rectangles = []
while True:
    prev_time = time.time()
    ret, frame_read = cap.read()

    if ret == False:
        break

    frame_rgb = cv2.cvtColor(frame_read, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb,
                               (outputWidth,
                                outputHeight),
                               interpolation=cv2.INTER_LINEAR)

    darknet.copy_image_from_bytes(darknet_image,frame_resized.tobytes())

    detections = darknet.detect_image(netMain, metaMain, darknet_image, thresh=0.50)
    image, area, rectangleB = cvDrawBoxes(detections, frame_resized)
    u = union(rectangleA, rectangleB) # 두 rectangle 의 합집합 영역
    i = intersection(rectangleA, rectangleB) # 두 rectangle 의 교집합 영역
    
    if not len(i) == 0: 
        u_area = u[1] * u[2]
        i_area = i[1] * i[2]
        
        try:
            iou = u_area / i_area # IoU = 합집합 면적 / 교집합 면적
        except ZeroDivisionError:
            print("ZeroDivision")
        print( iou )
        if iou > 1.0:
            # 테스트 용으로 우선은 교집합의 중심점을 라인으로 그리기 
            p1 = (int((rectangleB[0] + rectangleB[2]) / 2), int((rectangleB[1] + rectangleB[3]) / 2))
            cv2.line(image, p1, p1, (255, 0, 0), 20) 
    rectangleA = rectangleB
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    out.write(image)
    cv2.waitKey(3)
    
# print('calc start')
# for i in len(rectangles):
#     for j in len(rectangles):
#         u = union(rectangles[i], rectangles[j])
#         i = intersection(rectangles[i], rectangles[j])
#         if len(i) != 0:
#             u_area = u[1] * u[2]
#             i_area = i[1] * i[2]
#             try:
#                 iou = u_area / i_area
#                 print(iou)
#             except ZeroDivisionError:
#                 print("ZeroDivision")

#             if iou > 1.5:
#                 park_zones.append(rectangles[j])
#                 p1 = (int((rectangles[i][0] + rectangles[i][2]) / 2), int((rectangles[i][1] + rectangles[i][3]) / 2))
#                 cv2.line(temp_images[i], p1, p1, (255, 0, 0), 20)
#     image = cv2.cvtColor(temp_images[i], cv2.COLOR_BGR2RGB)
#     out.write(temp_images[i])
cap.release()
out.release()

Starting the YOLO loop...
0.99800796812749
1.0
1.0020242914979758
1.0020242914979758
1.0030364372469636
1.0010111223458038
1.1332053334667533
0.9989844951125282
1.0009587727708533
11.828107502799552
1.8728372520525145
2.994966141845329
1.1319621640237316
1.1352111240079055
1.1350746268656717
1.0045372050816697
1.0018115942028984
3.070977728438015
0.9998293196543031
37.362414883268485
1.1265909997461654
1.129551817091096
2.3994184810108017
3.6861108250077246
1.027237426035503
10.582093900336202
1.3067631923393406
2.689537629137933
1.9126475422082774
5.938707729468599
2.9396290910306555
2.9249909303576564
1.1985881864424375
5.19518383334817
1.6732935719019217
1.0005187747035573
1.0025
1.6120569318007254
1.0191518467852256
9.578592388210048
24.207249894063715
1.6075652188189196
6.154711326030868
4.737575977136417
1.9092702169625246
1.5736985658924296
4.1989375
2.384814453125
1.0023174971031286
1.637527686818202
1.003217821782178
3.5754921288518124
1.9960474308300395
1.0011467889908257
1.6

KeyboardInterrupt: 

In [ ]:
print('calc start')
for i in len(rectangles):
    for j in len(rectangles):
        u = union(rectangles[i], rectangles[j])
        i = intersection(rectangles[i], rectangles[j])
        if len(i) != 0:
            u_area = u[1] * u[2]
            i_area = i[1] * i[2]
            try:
                iou = u_area / i_area
                print(iou)
            except ZeroDivisionError:
                print("ZeroDivision")

            if iou > 1.5:
                park_zones.append(rectangles[j])
                p1 = (int((rectangles[i][0] + rectangles[i][2]) / 2), int((rectangles[i][1] + rectangles[i][3]) / 2))
                cv2.line(temp_images[i], p1, p1, (255, 0, 0), 20)
    image = cv2.cvtColor(temp_images[i], cv2.COLOR_BGR2RGB)
    out.write(temp_images[i])
    
cap.release()
out.release()